In [1]:
# import os, pyodbc, time, unidecode
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [2]:
# day_ago = 2
datenow = datetime.now().strftime("%Y%m%d")
# datenow_day_ago = ( datetime.now()-timedelta(day_ago) ).strftime("%Y%m%d")
# param_1 = f"'{datenow_day_ago}'"
param_2 = f"'20220401'"
param_3 = f"'{datenow}'"
param_4 = f"'20220405'"

query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI_v1] @Fromdate={param_3}, @Todate={param_3}"
FINAL = get_ms_df(sql=query)

In [3]:
FINAL.dtypes

Mã Công Ty/CN                    object
Công Ty/CN                       object
Ngày Chứng Từ            datetime64[ns]
Số Đơn Đặt Hàng                  object
MaHD                             object
Số Đơn Trả Hàng                  object
Ngày Trả Hàng                    object
Hóa Đơn                          object
Trạng Thái                       object
Mã KH DMS                        object
Mã KH Cũ                         object
Tên Khách Hàng                   object
THTT                             object
PMT                              object
Tên Vùng BH                      object
Tên Khu Vực                      object
Tên Tỉnh KH                      object
Tên Quận/HUyện                   object
Mã Kênh KH                       object
Tên Kênh KH                      object
Mã Kênh Phụ                      object
Tên Kênh Phụ                     object
Mã HCO                           object
Tên HCO                          object
Mã Phân Loại HCO                 object


In [7]:
FINAL.columns = cleancols(FINAL)

if str(FINAL.NgayGiaoHang.dtypes) == 'object' : FINAL.NgayGiaoHang = pd.to_datetime(FINAL.NgayGiaoHang, dayfirst=True)

if str(FINAL.NgayTraHang.dtypes) == 'object' : FINAL.NgayTraHang = pd.to_datetime(FINAL.NgayTraHang, dayfirst=True)

FINAL.NgayGiaoHang.fillna(datetime(1900, 1, 1), inplace=True)

FINAL['phanloaispcl'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, phanloai from d_nhom_sp where nhomsp='SPCL'"))
).fillna('Khác')

FINAL['nhomsp'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp IN ('SPCL', 'SP MOI') "))
).fillna('Khác')

FINAL['khuvucviettat'] = FINAL['TenKhuVuc'].map(
    df_to_dict(get_ps_df("select * from d_mkv_viet_tat"))
)

FINAL['chinhanh'] = FINAL['MaCongTyCN'].map(
    df_to_dict(get_ps_df("select * from d_chi_nhanh"))
)

FINAL['newhco'] = (FINAL['MaKenhPhu']+FINAL['MaPhanLoaiHCO']).map(
    df_to_dict(get_ps_df("SELECT concat(makenhphu, maphanloaihco) as concat, new_mahco FROM d_pl_hco"))
)

FINAL['phanam'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp='PHA NAM'"))).fillna('Merap')

FINAL['thang'] = FINAL['NgayChungTu'] + pd.offsets.Day() - pd.offsets.MonthBegin()

FINAL['inserted_at'] = datetime.now()

FINAL['PMT'] = np.where(FINAL['PMT'].isin(['B','C']),'TM','CK')

# BQ first

datenow_ = datetime.now().strftime("%Y-%m-%d")

In [8]:
FINAL.columns = lower_col(FINAL)

In [9]:
pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

In [10]:
# FINAL.columns

In [13]:
FINAL[checkdup(FINAL, 2, subset=pk)].to_clipboard()

In [13]:
if FINAL.shape[0] != 0 :

FINAL.columns = cleancols(FINAL)

if str(FINAL.NgayGiaoHang.dtypes) == 'object' : FINAL.NgayGiaoHang = pd.to_datetime(FINAL.NgayGiaoHang, dayfirst=True)

if str(FINAL.NgayTraHang.dtypes) == 'object' : FINAL.NgayTraHang = pd.to_datetime(FINAL.NgayTraHang, dayfirst=True)

FINAL.NgayGiaoHang.fillna(datetime(1900, 1, 1), inplace=True)

FINAL['phanloaispcl'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, phanloai from d_nhom_sp where nhomsp='SPCL'"))
).fillna('Khác')

FINAL['nhomsp'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp IN ('SPCL', 'SP MOI') "))
).fillna('Khác')

FINAL['khuvucviettat'] = FINAL['TenKhuVuc'].map(
    df_to_dict(get_ps_df("select * from d_mkv_viet_tat"))
)

FINAL['chinhanh'] = FINAL['MaCongTyCN'].map(
    df_to_dict(get_ps_df("select * from d_chi_nhanh"))
)

FINAL['newhco'] = (FINAL['MaKenhPhu']+FINAL['MaPhanLoaiHCO']).map(
    df_to_dict(get_ps_df("SELECT concat(makenhphu, maphanloaihco) as concat, new_mahco FROM d_pl_hco"))
)

FINAL['phanam'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp='PHA NAM'"))).fillna('Merap')

FINAL['thang'] = FINAL['NgayChungTu'] + pd.offsets.Day() - pd.offsets.MonthBegin()

FINAL['inserted_at'] = datetime.now()

FINAL['PMT'] = np.where(FINAL['PMT'].isin(['B','C']),'TM','CK')

# BQ first

datenow_ = datetime.now().strftime("%Y-%m-%d")

        sql =\
        f"""
        DELETE FROM biteam.f_sales where DATE(ngaychungtu)='{datenow_}'
        """
        execute_bq_query(sql)

        bq_values_insert(FINAL, "f_sales", 2)

        pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

        execute_values_upsert(FINAL, 'f_sales', pk=pk)
else:
    print('Not now')

In [14]:
# FINAL['inserted_at'] = datetime(2021,5,31)

In [15]:
# pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

In [16]:
bq_values_insert(FINAL, "f_sales", 2)

1it [00:12, 12.84s/it]


In [3]:
datenow = datetime.now().strftime("%Y-%m-%d")

In [4]:
datenow

'2022-04-05'

In [7]:
sql =\
f"""
DELETE FROM biteam.f_sales where DATE(ngaychungtu)='{datenow}'
"""

In [8]:
execute_bq_query(sql)

In [10]:
# df = get_bq_df("SELECT * FROM `spatial-vision-343005.biteam.f_sales` WHERE DATE(ngaychungtu) = '2022-04-05'")

In [11]:
# df.inserted_at.head()

In [25]:
FINAL['inserted_at'] = datetime(2021,5,31)

# pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

execute_values_upsert(FINAL, 'f_sales', pk=pk)

INSERT INTO f_sales(MaCongTyCN,CongTyCN,NgayChungTu,SoDonDatHang,MaHD,SoDonTraHang,NgayTraHang,HoaDon,TrangThai,MaKHDMS,MaKHCu,TenKhachHang,THTT,PMT,TenVungBH,TenKhuVuc,TenTinhKH,TenQuanHUyen,MaKenhKH,TenKenhKH,MaKenhPhu,TenKenhPhu,MaHCO,TenHCO,MaPhanLoaiHCO,TenPhanLoaiHCO,MaPhanHangHCO,TenPhanHangHCO,NhanHang,MaSanPham,TenSanPhamNB,TenSanPhamVietTat,SoLo,LineRef,SoLuong,DonGiaCoVAT,DoanhSoCoVAT,DonGiaChuaVAT,DoanhSoChuaVAT,NgayDatDon,NgayGiaoHang,MaNV,TenCVBH,TenQuanLyTT,TenQuanLyKhuVuc,TenQuanLyVung,MaNVGH,NguoiGiaohang,TrangThaiGiaoHang,DonViGiaoHang,TenNhaVanChuyen,KieuDonHang,MaKho,TenKho,phanloaispcl,nhomsp,khuvucviettat,chinhanh,newhco,phanam,thang,inserted_at) VALUES %s ON CONFLICT (macongtycn, ngaychungtu, sodondathang, masanpham, solo, lineref, soluong) DO UPDATE SET MaCongTyCN = excluded.MaCongTyCN, CongTyCN = excluded.CongTyCN, NgayChungTu = excluded.NgayChungTu, SoDonDatHang = excluded.SoDonDatHang, MaHD = excluded.MaHD, SoDonTraHang = excluded.SoDonTraHang, NgayTraHang = 

In [4]:
FINAL.columns = cleancols(FINAL)

FINAL.NgayGiaoHang.fillna(datetime(1900, 1, 1), inplace=True)

FINAL['phanloaispcl'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, phanloai from d_nhom_sp where nhomsp='SPCL'"))
).fillna('Khác')

FINAL['nhomsp'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp IN ('SPCL', 'SP MOI') "))
).fillna('Khác')

FINAL['khuvucviettat'] = FINAL['TenKhuVuc'].map(
    df_to_dict(get_ps_df("select * from d_mkv_viet_tat"))
)

FINAL['chinhanh'] = FINAL['MaCongTyCN'].map(
    df_to_dict(get_ps_df("select * from d_chi_nhanh"))
)

FINAL['newhco'] = (FINAL['MaKenhPhu']+FINAL['MaPhanLoaiHCO']).map(
    df_to_dict(get_ps_df("SELECT concat(makenhphu, maphanloaihco) as concat, new_mahco FROM d_pl_hco"))
)

FINAL['phanam'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp='PHA NAM'"))).fillna('Merap')

FINAL['thang'] = FINAL['NgayChungTu'] + pd.offsets.Day() - pd.offsets.MonthBegin()

FINAL['inserted_at'] = datetime(2022,2,7)

pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

execute_values_upsert(FINAL, 'f_sales', pk=pk)

INSERT INTO f_sales(MaCongTyCN,CongTyCN,NgayChungTu,SoDonDatHang,SoDonTraHang,NgayTraHang,HoaDon,TrangThai,MaKHThue,MaKHCu,TenKhachHang,TenVungBH,TenKhuVuc,TenTinhKH,TenQuanHUyen,MaKenhKH,TenKenhKH,MaKenhPhu,TenKenhPhu,MaHCO,TenHCO,MaPhanLoaiHCO,TenPhanLoaiHCO,MaPhanHangHCO,TenPhanHangHCO,NhanHang,MaSanPham,TenSanPhamNB,TenSanPhamVietTat,SoLo,LineRef,SoLuong,DonGiaCoVAT,DoanhSoCoVAT,DonGiaChuaVAT,DoanhSoChuaVAT,NgayDatDon,NgayGiaoHang,MaNV,TenCVBH,TenQuanLyTT,TenQuanLyKhuVuc,TenQuanLyVung,MaNVGH,NguoiGiaohang,TrangThaiGiaoHang,DonViGiaoHang,TenNhaVanChuyen,KieuDonHang,MaKho,TenKho,phanloaispcl,nhomsp,khuvucviettat,chinhanh,newhco,phanam,thang,inserted_at) VALUES %s ON CONFLICT (macongtycn, ngaychungtu, sodondathang, masanpham, solo, lineref, soluong) DO UPDATE SET MaCongTyCN = excluded.MaCongTyCN, CongTyCN = excluded.CongTyCN, NgayChungTu = excluded.NgayChungTu, SoDonDatHang = excluded.SoDonDatHang, SoDonTraHang = excluded.SoDonTraHang, NgayTraHang = excluded.NgayTraHang, HoaDon = excl

In [6]:
# FINAL.NgayGiaoHang.fillna(datetime(1900, 1, 1), inplace=True)

In [7]:
# vc(FINAL, 'NgayGiaoHang').to_clipboard()

In [8]:
# FINAL.dtypes

In [9]:
# FINAL.columns = lower_col(FINAL)

In [11]:
FINAL.columns

Index(['MaCongTyCN', 'CongTyCN', 'NgayChungTu', 'SoDonDatHang', 'SoDonTraHang',
       'NgayTraHang', 'HoaDon', 'TrangThai', 'MaKHThue', 'MaKHCu',
       'TenKhachHang', 'TenVungBH', 'TenKhuVuc', 'TenTinhKH', 'TenQuanHUyen',
       'MaKenhKH', 'TenKenhKH', 'MaKenhPhu', 'TenKenhPhu', 'MaHCO', 'TenHCO',
       'MaPhanLoaiHCO', 'TenPhanLoaiHCO', 'MaPhanHangHCO', 'TenPhanHangHCO',
       'NhanHang', 'MaSanPham', 'TenSanPhamNB', 'TenSanPhamVietTat', 'SoLo',
       'LineRef', 'SoLuong', 'DonGiaCoVAT', 'DoanhSoCoVAT', 'DonGiaChuaVAT',
       'DoanhSoChuaVAT', 'NgayDatDon', 'NgayGiaoHang', 'MaNV', 'TenCVBH',
       'TenQuanLyTT', 'TenQuanLyKhuVuc', 'TenQuanLyVung', 'MaNVGH',
       'NguoiGiaohang', 'TrangThaiGiaoHang', 'DonViGiaoHang',
       'TenNhaVanChuyen', 'KieuDonHang', 'MaKho', 'TenKho', 'phanloaispcl',
       'nhomsp', 'khuvucviettat', 'chinhanh', 'newhco', 'phanam', 'thang',
       'inserted_at'],
      dtype='object')

In [10]:
FINAL.columns

Index(['MaCongTyCN', 'CongTyCN', 'NgayChungTu', 'SoDonDatHang', 'SoDonTraHang',
       'NgayTraHang', 'HoaDon', 'TrangThai', 'MaKHThue', 'MaKHCu',
       'TenKhachHang', 'TenVungBH', 'TenKhuVuc', 'TenTinhKH', 'MaKenhKH',
       'TenKenhKH', 'MaKenhPhu', 'TenKenhPhu', 'MaHCO', 'TenHCO',
       'MaPhanLoaiHCO', 'TenPhanLoaiHCO', 'MaPhanHangHCO', 'TenPhanHangHCO',
       'NhanHang', 'MaSanPham', 'TenSanPhamNB', 'TenSanPhamVietTat', 'SoLo',
       'LineRef', 'SoLuong', 'DonGiaCoVAT', 'DoanhSoCoVAT', 'DonGiaChuaVAT',
       'DoanhSoChuaVAT', 'NgayDatDon', 'NgayGiaoHang', 'MaNV', 'TenCVBH',
       'TenQuanLyTT', 'TenQuanLyKhuVuc', 'TenQuanLyVung', 'MaNVGH',
       'NguoiGiaohang', 'TrangThaiGiaoHang', 'DonViGiaoHang',
       'TenNhaVanChuyen', 'KieuDonHang', 'MaKho', 'TenKho', 'phanloaispcl',
       'nhomsp', 'khuvucviettat', 'chinhanh', 'newhco', 'phanam', 'thang',
       'inserted_at'],
      dtype='object')

In [11]:
pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

In [12]:
# df_filter(FINAL, sodondathang='OO102021-00001').to_clipboard()

In [13]:
execute_values_upsert(FINAL, 'f_sales', pk=pk)

INSERT INTO f_sales(MaCongTyCN,CongTyCN,NgayChungTu,SoDonDatHang,SoDonTraHang,NgayTraHang,HoaDon,TrangThai,MaKHThue,MaKHCu,TenKhachHang,TenVungBH,TenKhuVuc,TenTinhKH,MaKenhKH,TenKenhKH,MaKenhPhu,TenKenhPhu,MaHCO,TenHCO,MaPhanLoaiHCO,TenPhanLoaiHCO,MaPhanHangHCO,TenPhanHangHCO,NhanHang,MaSanPham,TenSanPhamNB,TenSanPhamVietTat,SoLo,LineRef,SoLuong,DonGiaCoVAT,DoanhSoCoVAT,DonGiaChuaVAT,DoanhSoChuaVAT,NgayDatDon,NgayGiaoHang,MaNV,TenCVBH,TenQuanLyTT,TenQuanLyKhuVuc,TenQuanLyVung,MaNVGH,NguoiGiaohang,TrangThaiGiaoHang,DonViGiaoHang,TenNhaVanChuyen,KieuDonHang,MaKho,TenKho,phanloaispcl,nhomsp,khuvucviettat,chinhanh,newhco,phanam,thang,inserted_at) VALUES %s ON CONFLICT (macongtycn, ngaychungtu, sodondathang, masanpham, solo, lineref, soluong) DO UPDATE SET MaCongTyCN = excluded.MaCongTyCN, CongTyCN = excluded.CongTyCN, NgayChungTu = excluded.NgayChungTu, SoDonDatHang = excluded.SoDonDatHang, SoDonTraHang = excluded.SoDonTraHang, NgayTraHang = excluded.NgayTraHang, HoaDon = excluded.HoaDon, 

In [39]:
# FINAL2 = FINAL.head()

In [40]:
# FINAL2.to_csv('FINAL.csv')

In [24]:
############# ENDDDDDDDDDDD #################

In [2]:
df_nhansu = pd.read_csv('datanhansu.csv')

In [3]:
df_nhansu.columns = cleancols(df_nhansu)

In [4]:
print_df_schema(df_nhansu)

STT BIGINT NULL,
PHONG VARCHAR(100) NULL,
BP VARCHAR(100) NULL,
MaNVCRSCRSS VARCHAR(100) NULL,
HOTEN VARCHAR(100) NULL,
HOTENKHONGDAU VARCHAR(100) NULL,
CHUCDANH VARCHAR(100) NULL,
TINHTRANG VARCHAR(100) NULL,
Ngayvaocongty VARCHAR(100) NULL,
KV VARCHAR(100) NULL,
SDT VARCHAR(100) NULL,
EMAIL VARCHAR(100) NULL,
MANVQLTT VARCHAR(100) NULL,
QUANLYTRUCTIEP VARCHAR(100) NULL,
MANVQLV VARCHAR(100) NULL,
QLKHUVUC VARCHAR(100) NULL,
MANVQLV1 VARCHAR(100) NULL,
QLVUNG VARCHAR(100) NULL,


In [10]:
df_nhansu.columns

Index(['STT', 'PHONG', 'BP', 'MaNVCRSCRSS', 'HOTEN', 'HOTENKHONGDAU',
       'CHUCDANH', 'TINHTRANG', 'Ngayvaocongty', 'KV', 'SDT', 'EMAIL',
       'MANVQLTT', 'QUANLYTRUCTIEP', 'MANVQLV', 'QLKHUVUC', 'MANVQLV.1',
       'QLVUNG'],
      dtype='object')

In [5]:
insert_df_to_postgres(
postgres_conf, tbl_name="nhansu",
df=df_nhansu, primary_keys=['manvcrscrss'], schema="biteam")

In [2]:
# df_dict = pd.read_csv('ten_tat_mkv.csv', index_col='TenKhuVuc')
df_mkv_viet_tat = pd.read_csv('ten_tat_mkv.csv')

In [3]:
print_df_schema(df_mkv_viet_tat)

TenKhuVuc VARCHAR(100) NULL,
KhuVuc VARCHAR(100) NULL,


In [7]:
x = [x.lower() for x in df_mkv_viet_tat.columns]
df_mkv_viet_tat.columns = x
insert_df_to_postgres(df=df_mkv_viet_tat, postgres_conf=postgres_conf, tbl_name='mkv_viet_tat',schema="biteam", primary_keys=['tenkhuvuc'])

In [1]:
t1 = time.perf_counter()
now = datetime.now()
print('start program %s' % now)
print('current location %s' % os.getcwd())

# Server config
server = '115.165.164.234'
driver = 'SQL Server'
db1 = 'PhaNam_eSales_PRO'
tcon = 'no'
uname = 'duyvq'
pword = '123VanQuangDuy'
cnxn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}',
                      host=server, database=db1, trusted_connection=tcon,
                      user=uname, password=pword)
cursor = cnxn.cursor()

# Load df from server
print('Load df from server', datetime.now())
datenow = datetime.now().strftime("%Y%m%d")
param_1 = f"'20210901'"
param_2 = f"'20210930'"
query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI] @Fromdate={param_1}, @Todate={param_2}"
PATH = "D:\\data_sale\\ipython_file\\data_raw_thang\\"
DATAPATH = PATH + "data\\"
FILE = f"data\\raw_data_old_{param_2}.csv"

# df1 = pd.read_sql(query, cnxn)
cursor.close()
cnxn.close()
# df1.to_csv(PATH + FILE)

start program 2021-10-08 12:32:26.251163
current location d:\data_sale\ipython_file\data_raw_thang
Load df from server 2021-10-08 12:32:26.362819


In [4]:
PATH = "D:\\data_sale\\ipython_file\\data_raw_thang\\"
DATAPATH = PATH + "data\\"
FILE = f"data\\raw_data_old_{param_2}.csv"

NameError: name 'param_2' is not defined

In [5]:
def csv_reader(filename):
    df_from_csv = pd.read_csv(filename, parse_dates=['Ngày Chứng Từ'], dayfirst=True,
                              dtype={'Hóa Đơn':'str'})
    return df_from_csv
df = csv_reader(DATAPATH+'raw_data_old_20210930.csv')
df.columns = cleancols(df)

C:\Users\DELL\AppData\Local\Temp/ipykernel_13484/4228445815.py:5: DtypeWarning: Columns (5,58) have mixed types.Specify dtype option on import or set low_memory=False.
  df = csv_reader(DATAPATH+'raw_data_old_20210930.csv')


In [6]:
df_mdh_dvvc = df[['MaCongTyCN','SoDonDatHang', 'DonViGiaoHang']].copy()
df_mdh_dvvc['MaCongTyCN_SoDonDatHang'] = df['MaCongTyCN']+df['SoDonDatHang']
df_mdh_dvvc_2 = df_mdh_dvvc[['MaCongTyCN_SoDonDatHang', 'DonViGiaoHang']]
df_mdh_dvvc_2 = dropdup(df_mdh_dvvc_2,1)
mdh_dvvc_dict= df_mdh_dvvc_2.set_index('MaCongTyCN_SoDonDatHang').to_dict()['DonViGiaoHang']
x = df['DonViGiaoHang']
df['DonViGiaoHang'] = np.where(x.isna(), (df['MaCongTyCN']+df['SoDonDatHang']).map(mdh_dvvc_dict),x)

In [13]:
t1 = time.perf_counter()

print("Done merging")
# Transforming df
print('Transforming df', datetime.now())
df.replace(np.nan, '', inplace=True)
print(df.shape[0])

df_dict = pd.read_csv('ten_tat_mkv.csv', index_col='TenKhuVuc')
khuvuc_dict = df_dict.to_dict()['KhuVuc']

df['TenKhuVucVietTat'] = df['TenKhuVuc'].map(khuvuc_dict)

# Google Sheet connection
print('Google Sheet connection & join data', datetime.now())
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file']
jsonfile = 'D:\\data_sale\\datateam1599968716114-6f9f144b4262.json'
credentials = service_account.Credentials.from_service_account_file(jsonfile, scopes=scopes)
service = discovery.build('drive', 'v3', credentials=credentials)
# df_product
url_product='https://drive.google.com/file/d/15ClbdtRJhqF6bIDNgRLqI1wAytIoPWWR/view?usp=sharing'
file_id_product=url_product.split('/')[-2]
dwn_url_product='https://drive.google.com/uc?id=' + file_id_product
df_product = pd.read_csv(dwn_url_product)
df = df.merge(df_product[[' Mã Sản Phẩm',' Ngành Hàng']], how = 'left',left_on='MaSanPham', right_on=' Mã Sản Phẩm',suffixes=('_left', '_right'), validate="m:1")


# df spcl_mapping -> Phan Loai
url_spcl = 'https://drive.google.com/file/d/190OCbFpxOT2KCjwhjC5nSbdcmA7ScbdW/view?usp=sharing'
file_id_spcl=url_spcl.split('/')[-2]
dwn_url_spcl = 'https://drive.google.com/uc?id=' + file_id_spcl
df_spcl = pd.read_csv(dwn_url_spcl)
df = df.merge(df_spcl, how='left', left_on='MaSanPham', right_on='MaSanPham',suffixes=('_left', '_right'), validate="m:1")
df.rename(columns={'PhanLoai': 'Nhom SPCL'}, inplace=True)

# MA HCO 2
df['MaKenhPhu_MaHCO'] = df['MaKenhPhu']+df['MaPhanLoaiHCO']
url_phanloaihco = 'https://drive.google.com/file/d/1lLdheM4T5Rp5Ht4SMjxAtm4v61wwx9Xr/view?usp=sharing'
file_id_phanloaihco=url_phanloaihco.split('/')[-2]
dwn_url_phanloaihco = 'https://drive.google.com/uc?id=' + file_id_phanloaihco
df_phanloaihco = pd.read_csv(dwn_url_phanloaihco)
df = df.merge(df_phanloaihco[['ConCat','Ma HCO 2']], how='left', left_on='MaKenhPhu_MaHCO', right_on='ConCat',suffixes=('_left', '_right'), validate="m:1")
df.rename(columns={'Ma HCO 2': 'PL HCO'}, inplace=True)

# df_clean_data
df = df[df['MaSanPham'].str[:1] != 'V']
df = df[df['MaKenhKH'] != 'NB']
df['SDDH-CN'] = df['SoDonDatHang']+'.'+df['MaCongTyCN']

print(df.shape[0])

# NhomSPCL
x = np.array(df['MaSanPham'])
spcl_inlist = ['EH111',	'EH086', 'EH104', 'EH106', 'EH103', 'EH109', 'EH115', 'EH094', 'EH092', 'EH102', 'EH116',
               'EH087', 'EH108', 'EH091', 'EH112', 'EH101', 'EH117', 'VITOL 0.10%', 'EH125', 'EH124', 'OH082']
spm_inlist = [' EH115 ', 'OH072', 'OH076']
condlist = [np.isin(x, spcl_inlist), np.isin(x, spm_inlist)]
choicelist = ['SPCL', 'SP MOI']
default = 'KHAC'
df['NhomSP'] = np.select(condlist, choicelist, default=default)

# MaMien(B/N)
x = np.array(df['MaVungBH'])
condlist = [np.isin(x, ['BB', 'HN'])]
choicelist = ['MB']
default = 'MN'
df['MaMien(B/N)'] = np.select(condlist, choicelist, default=default)

df1 = df['MaNVGH'].copy()
df1.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df['MaNVGH'] = df1

df['NVBH_Ao'] = np.where(df['MaNV'].isin(['MA001', 'MA002', 'QUYNHPTA']), 0, 1)
df['NVGH_Ao'] = np.where(df['MaNVGH'].isna(), 0, 1)

df['DoanhSoBanHangChuaVAT'] = df['DoanhSoChuaVAT'] * df['NVBH_Ao']
df['DoanhSoGiaoHangChuaVAT'] = df['DoanhSoChuaVAT'] * df['NVGH_Ao']

df['SoKH'] = np.where(checkdup(df,1, 'MaKHCu'),0,1)
df['SoDH'] = np.where(checkdup(df,1, 'SDDH-CN'),0,1)


# Merap/Phanam
phanam_inlist = ['EH072', 'OH057', 'OH032', 'OH047','OH016','OH058','OH060','OH061',
          'OH062','OH063','OH064','OH065','OH070','OH069','OH066','OH068','OH067','EH105','OH071','OH079','OH081']

x = np.array(df['MaSanPham'])
condlist = [np.isin(x, phanam_inlist)]
choicelist = ['Pha Nam']
default = 'Merap'
df['Merap/PhaNam'] = np.select(condlist, choicelist, default=default)

#VanPhong
df['ChiNhanh'] = df['CongTyCN'].str.split(' ').apply(lambda x: x[-2:]).apply(lambda x: ''+' '.join(x))

cols = df.columns
unwanted_cols = {'DiaChiKH','SoHopDong','MaKenhPhu_MaHCO','ConCat',' Mã Sản Phẩm', ' Ngành Hàng','Unnamed0','MaKHThue','TenKHThue','DiaChiKHThue','MaSoThue','MaKhuVuc','MaTinhKH','MaQuan/HUyen','MaPhanHangHCO','TenPhanHangHCO','TenSanPhamNB'}
cols = [ele for ele in cols if ele not in unwanted_cols]

df=df[cols]

#df to csv
df.to_csv("raw_data.csv", index=False)


now_final = datetime.now()
print("finally imported sales data %s" % now_final)
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Done merging
Transforming df 2021-10-28 16:06:17.707622
61014
Google Sheet connection & join data 2021-10-28 16:06:17.832142
61014
finally imported sales data 2021-10-28 16:06:25.409195
Finished in 7.702465800000027 seconds
